## pandas pivot_table()

In [1]:
import pandas as pd


def load_data():
    return pd.read_csv('coffee_sales.csv', parse_dates=['order_date'])
df=load_data()
df

,order_date,market,region,product_category,product,cost,inventory,net_profit,sales
0,2010-01-01,Wholesale,Central,Coffee,Amaretto,89,777,94,219
1,2010-01-01,Wholesale,Central,Coffee,Columbian,83,623,68,190
2,2010-01-01,Wholesale,Central,Coffee,Decaf Irish Cream,95,821,101,234
3,2010-01-01,Wholesale,Central,Tea,Green Tea,44,623,30,100
4,2010-01-01,Wholesale,Central,Espresso,Caffe Mocha,54,456,54,134
...,...,...,...,...,...,...,...,...,...
4243,2011-12-01,Retail,West,Espresso,Caffe Latte,24,567,19,60
4244,2011-12-01,Retail,West,Espresso,Caffe Mocha,65,403,34,155
4245,2011-12-01,Retail,West,Espresso,Decaf Espresso,80,1079,76,188
4246,2011-12-01,Retail,West,Coffee,Columbian,72,461,86,188


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4248 entries, 0 to 4247
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_date        4248 non-null   datetime64[ns]
 1   market            4248 non-null   object        
 2   region            4248 non-null   object        
 3   product_category  4248 non-null   object        
 4   product           4248 non-null   object        
 5   cost              4248 non-null   int64         
 6   inventory         4248 non-null   int64         
 7   net_profit        4248 non-null   int64         
 8   sales             4248 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 298.8+ KB


In [3]:
# Simplest pivot table

df1=df.pivot_table(index='region')
df1.round(2)

,cost,inventory,net_profit,sales
region,,,,
Central,84.16,742.65,69.83,197.21
East,85.42,778.19,66.69,201.10
South,65.33,478.27,48.33,154.65
West,93.61,872.63,55.06,202.58


In [4]:
# with multiple indexes
df2=df.pivot_table(index=['region', 'product_category'])
df2.round(2)

cost  inventory  net_profit   sales
region  product_category                                       
Central Coffee             74.39     680.78       60.58  179.90
        Espresso           89.67     882.09       81.60  207.30
        Herbal tea         87.07     763.95       73.68  202.05
        Tea                87.69     672.53       66.46  203.50
East    Coffee            110.98     580.79      184.48  337.15
        Espresso           95.60     767.13       26.02  201.65
        Herbal tea        101.24     963.21       29.74  191.49
        Tea                46.96     762.48       58.93  121.88
South   Coffee             78.92     803.69       60.95  173.22
        Espresso           62.31     184.65       52.10  156.22
        Herbal tea         56.27     593.30       30.06  133.73
West    Coffee             90.81     931.58       27.96  185.41
        Espresso           84.13     828.93       66.31  194.19
        Herbal tea         98.99     802.28       84.30  231.69
        Tea               100.84     926.21       41.94  200.61

In [5]:
# specify values and perform aggregation

df.pivot_table(index='region', values='sales').round(2)

,sales
region,
Central,197.21
East,201.10
South,154.65
West,202.58


In [6]:
df3=df.pivot_table(index='region', values='sales', aggfunc='sum')
df3

,sales
region,
Central,265045
East,178576
South,103926
West,272264


In [7]:
df.pivot_table(index='region', values='sales', aggfunc={'sales':'sum'})

,sales
region,
Central,265045
East,178576
South,103926
West,272264


In [8]:
df4=df.pivot_table(index='region', values='sales', aggfunc=['sum', 'count'])
df4

,sum,count
,sales,sales
region,,
Central,265045,1344
East,178576,888
South,103926,672
West,272264,1344


In [9]:
df5=df4.applymap(lambda x:'{:,}'.format(x))
df5

,sum,count
,sales,sales
region,,
Central,"265,045","1,344"
East,"178,576",888
South,"103,926",672
West,"272,264","1,344"


In [10]:
df5['sum']['sales'].apply(lambda x:print(type(x)))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


region
Central    None
East       None
South      None
West       None
Name: sales, dtype: object

In [11]:
# break down using columns
df.pivot_table(index='region', values='sales', aggfunc='sum', columns='product_category')

product_category,Coffee,Espresso,Herbal tea,Tea
region,,,,
Central,69080.0,59701.0,67888.0,68376.0
East,56641.0,48397.0,41361.0,32177.0
South,33259.0,44990.0,25677.0,NaN
West,57848.0,69908.0,72288.0,72220.0


In [12]:
# replace missing value
df.pivot_table(index='region', values='sales', aggfunc='sum', columns='product_category', fill_value=0)

product_category,Coffee,Espresso,Herbal tea,Tea
region,,,,
Central,69080,59701,67888,68376
East,56641,48397,41361,32177
South,33259,44990,25677,0
West,57848,69908,72288,72220


In [13]:
# Move product_category to the index to get a better view

df6=df.pivot_table(index=['region', 'product_category'], values='sales', aggfunc='sum', fill_value=0)
df6=df6['sales'].apply(lambda x:"{:,}".format(x))
df6

region   product_category
Central  Coffee              69,080
         Espresso            59,701
         Herbal tea          67,888
         Tea                 68,376
East     Coffee              56,641
         Espresso            48,397
         Herbal tea          41,361
         Tea                 32,177
South    Coffee              33,259
         Espresso            44,990
         Herbal tea          25,677
West     Coffee              57,848
         Espresso            69,908
         Herbal tea          72,288
         Tea                 72,220
Name: sales, dtype: object

In [14]:
df7=pd.DataFrame(df6)
df7

sales
region  product_category        
Central Coffee            69,080
        Espresso          59,701
        Herbal tea        67,888
        Tea               68,376
East    Coffee            56,641
        Espresso          48,397
        Herbal tea        41,361
        Tea               32,177
South   Coffee            33,259
        Espresso          44,990
        Herbal tea        25,677
West    Coffee            57,848
        Espresso          69,908
        Herbal tea        72,288
        Tea               72,220

In [15]:
# shows total (margins)

df8=df.pivot_table(index=['region', 'product_category'], 
                   values=['sales', 'cost'], 
                   aggfunc='sum', 
                   fill_value=0, 
                   margins=True)
df8.applymap(lambda x:f'{x:,}')

cost    sales
region  product_category                  
Central Coffee             28,566   69,080
        Espresso           25,824   59,701
        Herbal tea         29,254   67,888
        Tea                29,464   68,376
East    Coffee             18,644   56,641
        Espresso           22,944   48,397
        Herbal tea         21,868   41,361
        Tea                12,398   32,177
South   Coffee             15,152   33,259
        Espresso           17,944   44,990
        Herbal tea         10,804   25,677
West    Coffee             28,334   57,848
        Espresso           30,288   69,908
        Herbal tea         30,884   72,288
        Tea                36,304   72,220
All                       358,672  819,811

In [16]:
# Generate a monthly report

month_gp=pd.Grouper(key='order_date', freq='M')
cond=df['order_date'].dt.year==2010

df9=df[cond].pivot_table(index=['region', 'product_category'],
                    columns=month_gp,
                    values='sales')

# df9.applymap(lambda x:f"${x:,}")

In [17]:
df9.columns

DatetimeIndex(['2010-01-31', '2010-02-28', '2010-03-31', '2010-04-30',
               '2010-05-31', '2010-06-30', '2010-07-31', '2010-08-31',
               '2010-09-30', '2010-10-31', '2010-11-30', '2010-12-31'],
              dtype='datetime64[ns]', name='order_date', freq='M')

In [18]:
months=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

zipped=zip(months, df9.columns)

for month, col in zipped:
    print(month, col)

Jan 2010-01-31 00:00:00
Feb 2010-02-28 00:00:00
Mar 2010-03-31 00:00:00
Apr 2010-04-30 00:00:00
May 2010-05-31 00:00:00
Jun 2010-06-30 00:00:00
Jul 2010-07-31 00:00:00
Aug 2010-08-31 00:00:00
Sep 2010-09-30 00:00:00
Oct 2010-10-31 00:00:00
Nov 2010-11-30 00:00:00
Dec 2010-12-31 00:00:00


In [19]:
df9.columns=months
df9=df9.round(2)
df10=df9.applymap(lambda x:f'${x:,}')
df10

Jan      Feb      Mar      Apr      May  \
region  product_category                                                
Central Coffee            $166.44  $169.25  $168.94  $175.94  $179.88   
        Espresso           $197.5  $202.67  $205.58  $203.83  $208.08   
        Herbal tea        $190.29  $193.14  $192.21  $195.36  $199.64   
        Tea               $189.21  $189.93   $193.0  $198.14  $202.86   
East    Coffee            $300.71  $294.43  $303.71  $322.57  $335.29   
        Espresso           $185.3   $196.6   $190.7   $193.5   $189.6   
        Herbal tea        $178.78  $180.11   $182.0  $194.78  $197.67   
        Tea               $110.27  $115.64  $113.64  $115.18  $119.91   
South   Coffee            $150.88   $157.5  $158.12   $160.0  $171.12   
        Espresso           $147.5  $150.25  $148.42  $149.17  $154.08   
        Herbal tea        $124.88  $127.38  $126.12  $124.38  $122.25   
West    Coffee            $179.92  $176.77  $177.92  $179.38  $180.85   
        Espresso          $181.53  $183.93   $186.4  $186.53   $188.6   
        Herbal tea        $200.15  $207.85  $210.69  $221.85  $228.08   
        Tea               $185.93  $188.53  $190.53  $192.07   $200.0   

                              Jun      Jul      Aug      Sep      Oct  \
region  product_category                                                
Central Coffee             $188.0  $196.75  $188.25  $170.88  $168.38   
        Espresso          $208.08   $213.0  $212.83   $199.5  $199.25   
        Herbal tea        $207.71  $212.57   $211.5  $196.21  $195.29   
        Tea                $204.5   $208.0  $207.79  $203.64  $197.29   
East    Coffee             $375.0  $390.71  $381.86  $337.43  $299.14   
        Espresso           $207.1   $199.2   $198.4   $188.7   $212.4   
        Herbal tea        $199.44   $214.0  $186.33   $171.0  $171.56   
        Tea               $127.64  $126.82  $133.55  $121.55  $121.73   
South   Coffee            $178.62  $186.12  $197.38  $173.62  $167.62   
        Espresso           $158.0  $160.25  $160.25  $153.67  $151.67   
        Herbal tea         $127.0   $129.5  $142.38  $129.12  $139.88   
West    Coffee            $191.92  $195.46  $194.08  $181.46  $171.85   
        Espresso          $193.13  $199.47  $205.13  $187.73  $190.47   
        Herbal tea         $241.0   $260.0  $260.46  $219.23  $221.46   
        Tea               $205.87  $210.67  $210.87  $196.67  $194.47   

                              Nov      Dec  
region  product_category                    
Central Coffee             $164.5  $175.81  
        Espresso          $190.17  $193.33  
        Herbal tea        $187.93  $190.64  
        Tea               $190.71  $204.29  
East    Coffee            $295.14  $326.86  
        Espresso           $207.6   $198.1  
        Herbal tea        $178.33   $195.0  
        Tea               $111.91  $113.55  
South   Coffee            $161.75  $172.25  
        Espresso          $149.67  $151.08  
        Herbal tea        $139.38   $137.5  
West    Coffee            $171.38  $176.38  
        Espresso          $185.87   $191.2  
        Herbal tea        $215.85  $235.77  
        Tea               $185.53  $194.73

## Pandas.melt()

If data contains values in the column headers instead of variables, we can melt or unpivot the wide form data to long form so the resahped data becomes more computer-friendly form. <br>
```python
pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)
```

In [20]:
import pandas as pd

# creating a dataframe
df = pd.DataFrame({'Name': {0: 'John', 1: 'Bob', 2: 'Shiela'},
                'Course': {0: 'Masters', 1: 'Graduate', 2: 'Graduate'},
                'Age': {0: 27, 1: 23, 2: 21}})
                
df

,Name,Course,Age
0,John,Masters,27
1,Bob,Graduate,23
2,Shiela,Graduate,21


In [21]:
# Name is id_vars and Course is value_vars
pd.melt(df, id_vars =['Name'], value_vars =['Course'])

,Name,variable,value
0,John,Course,Masters
1,Bob,Course,Graduate
2,Shiela,Course,Graduate


In [22]:
# multiple unpivot columns
pd.melt(df, id_vars =['Name'], value_vars =['Course', 'Age'])

,Name,variable,value
0,John,Course,Masters
1,Bob,Course,Graduate
2,Shiela,Course,Graduate
3,John,Age,27
4,Bob,Age,23
5,Shiela,Age,21


In [23]:
# Names of ‘variable’ and ‘value’ columns can be customized
pd.melt(df, id_vars =['Name'], value_vars =['Course'],
        var_name ='ChangedVarname', value_name ='ChangedValname')


,Name,ChangedVarname,ChangedValname
0,John,Course,Masters
1,Bob,Course,Graduate
2,Shiela,Course,Graduate


In [24]:
# https://lifewithdata.com/2022/02/21/pandas-pd-melt-how-to-unpivot-in-pandas/
import pandas as pd
df=pd.DataFrame({'nation':['South Korea', 'China', 'Canada'],
                'gold':[24, 10, 9],
                'silver':[13, 15, 12],
                'bronz':[11, 8, 12]})
df

,nation,gold,silver,bronz
0,South Korea,24,13,11
1,China,10,15,8
2,Canada,9,12,12


In [25]:
# keep nation as it is
pd.melt(df, id_vars='nation')

,nation,variable,value
0,South Korea,gold,24
1,China,gold,10
2,Canada,gold,9
3,South Korea,silver,13
4,China,silver,15
5,Canada,silver,12
6,South Korea,bronz,11
7,China,bronz,8
8,Canada,bronz,12


In [26]:
pd.melt(df, id_vars='nation', var_name='Medals', value_name='Count')

,nation,Medals,Count
0,South Korea,gold,24
1,China,gold,10
2,Canada,gold,9
3,South Korea,silver,13
4,China,silver,15
5,Canada,silver,12
6,South Korea,bronz,11
7,China,bronz,8
8,Canada,bronz,12


In [27]:
# keep more than one column as it is and melt the rest of the columns
df = pd.read_csv("https://raw.githubusercontent.com/bprasad26/lwd/master/data/billboard.csv")
df.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
pd.melt(df, id_vars=['year', 'artist', 'track', 'time', 'date.entered'], var_name='week')

,year,artist,track,time,date.entered,week,value
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0
...,...,...,...,...,...,...,...
24087,2000,Yankee Grey,Another Nine Minutes,3:10,2000-04-29,wk76,NaN
24088,2000,"Yearwood, Trisha",Real Live Woman,3:55,2000-04-01,wk76,NaN
24089,2000,Ying Yang Twins,Whistle While You Tw...,4:19,2000-03-18,wk76,NaN
24090,2000,Zombie Nation,Kernkraft 400,3:30,2000-09-02,wk76,NaN
